In [42]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori, association_rules

In [112]:
df = pd.read_csv("BreadBasket.csv")
df

,Date,Time,Transaction,Item
0,2016-10-30,09:58:11,1,Bread
1,2016-10-30,10:05:34,2,Scandinavian
2,2016-10-30,10:05:34,2,Scandinavian
3,2016-10-30,10:07:57,3,Hot chocolate
4,2016-10-30,10:07:57,3,Jam
...,...,...,...,...
21288,2017-04-09,14:32:58,9682,Coffee
21289,2017-04-09,14:32:58,9682,Tea
21290,2017-04-09,14:57:06,9683,Coffee
21291,2017-04-09,14:57:06,9683,Pastry


In [113]:
df.drop('Date', inplace=True, axis=1)
df.drop('Time', inplace=True, axis=1)
df

,Transaction,Item
0,1,Bread
1,2,Scandinavian
2,2,Scandinavian
3,3,Hot chocolate
4,3,Jam
...,...,...
21288,9682,Coffee
21289,9682,Tea
21290,9683,Coffee
21291,9683,Pastry


In [114]:
onehot = df.pivot_table(index='Transaction', columns='Item', aggfunc=len, fill_value=0)
onehot.head()

Item,Adjustment,Afternoon with the baker,Alfajores,Argentina Night,Art Tray,Bacon,Baguette,Bakewell,Bare Popcorn,Basket,...,The BART,The Nomad,Tiffin,Toast,Truffles,Tshirt,Valentine's card,Vegan Feast,Vegan mincepie,Victorian Sponge
Transaction,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [136]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1
basket = onehot.applymap(encode_units)

#dari file yang saya dapat kaggle tertulis keterangan seperti dibawah
#karena saya kurang yakin dan menurut saya ambigu jadi saya hapus

#the Item contains "Adjustment", "NONE", and "Afternoon with the baker". 
#While the entries of "Adjustment" and "NONE" are straight forward, 
#"Afternoon with the baker" may be a real purchase.

#menghapus kolom NONE, Adjustment, Afternoon with the baker.
basket.drop('NONE', inplace=True, axis=1)
basket.drop('Adjustment', inplace=True, axis=1)
basket.drop('Afternoon with the baker', inplace=True, axis=1)

basket.head()

Item,Alfajores,Argentina Night,Art Tray,Bacon,Baguette,Bakewell,Bare Popcorn,Basket,Bowl Nic Pitt,Bread,...,The BART,The Nomad,Tiffin,Toast,Truffles,Tshirt,Valentine's card,Vegan Feast,Vegan mincepie,Victorian Sponge
Transaction,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [145]:
frequent_itemsets = apriori(basket, min_support=0.03, use_colnames= True)
frequent_itemsets

,support,itemsets
0,0.036093,(Alfajores)
1,0.324940,(Bread)
2,0.039765,(Brownie)
3,0.103137,(Cake)
4,0.475081,(Coffee)
5,0.054034,(Cookies)
6,0.038926,(Farm House)
7,0.057916,(Hot chocolate)
8,0.038296,(Juice)
9,0.061379,(Medialuna)


In [151]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=0.7)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Cake),(Coffee),0.103137,0.475081,0.054349,0.526958,1.109196,0.005350,1.109667
1,(Coffee),(Cake),0.475081,0.103137,0.054349,0.114399,1.109196,0.005350,1.012717
2,(Coffee),(Medialuna),0.475081,0.061379,0.034939,0.073542,1.198175,0.005779,1.013129
3,(Medialuna),(Coffee),0.061379,0.475081,0.034939,0.569231,1.198175,0.005779,1.218561
4,(Pastry),(Coffee),0.085510,0.475081,0.047214,0.552147,1.162216,0.006590,1.172079
5,(Coffee),(Pastry),0.475081,0.085510,0.047214,0.099382,1.162216,0.006590,1.015402
6,(Sandwich),(Coffee),0.071346,0.475081,0.037981,0.532353,1.120551,0.004086,1.122468
7,(Coffee),(Sandwich),0.475081,0.071346,0.037981,0.079947,1.120551,0.004086,1.009348
8,(Tea),(Coffee),0.141643,0.475081,0.049523,0.349630,0.735936,-0.017769,0.807107
9,(Coffee),(Tea),0.475081,0.141643,0.049523,0.104240,0.735936,-0.017769,0.958245


In [155]:
rules[ (rules['lift'] >= 0.7) & (rules['confidence'] >= 0.3) ]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Cake),(Coffee),0.103137,0.475081,0.054349,0.526958,1.109196,0.005350,1.109667
3,(Medialuna),(Coffee),0.061379,0.475081,0.034939,0.569231,1.198175,0.005779,1.218561
4,(Pastry),(Coffee),0.085510,0.475081,0.047214,0.552147,1.162216,0.006590,1.172079
6,(Sandwich),(Coffee),0.071346,0.475081,0.037981,0.532353,1.120551,0.004086,1.122468
8,(Tea),(Coffee),0.141643,0.475081,0.049523,0.349630,0.735936,-0.017769,0.807107
